# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [5]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [6]:
encoded[:100]

array([73, 66, 51, 20, 13, 54, 47, 23, 45, 53, 53, 53, 59, 51, 20, 20, 30,
       23, 82, 51, 33, 77, 16, 77, 54, 68, 23, 51, 47, 54, 23, 51, 16, 16,
       23, 51, 16, 77, 80, 54, 48, 23, 54, 32, 54, 47, 30, 23, 15, 37, 66,
       51, 20, 20, 30, 23, 82, 51, 33, 77, 16, 30, 23, 77, 68, 23, 15, 37,
       66, 51, 20, 20, 30, 23, 77, 37, 23, 77, 13, 68, 23, 25, 69, 37, 53,
       69, 51, 30,  5, 53, 53, 58, 32, 54, 47, 30, 13, 66, 77, 37])

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [7]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [8]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [27]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## TODO: Get the number of batches we can make
    n_batches = len(arr)//(batch_size*seq_length)
    
    ## TODO: Keep only enough characters to make full batches
    arr = arr[:n_batches*(batch_size*seq_length)]
    
    ## TODO: Reshape into batch_size rows
    arr = arr.reshape((batch_size,-1))
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:,n:n+seq_length]
        # The targets, shifted by one
        if n < arr.shape[1]-seq_length:
            y = arr[:,n+1:n+seq_length+1]
        else:
            y = x.copy()
            y[:,:-1], y[:,-1] = arr[:,n+1:n+seq_length+1], arr[:, 0]
        yield x, y

### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [28]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [32]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[73 66 51 20 13 54 47 23 45 53]
 [68 25 37 23 13 66 51 13 23 51]
 [54 37 81 23 25 47 23 51 23 82]
 [68 23 13 66 54 23 56 66 77 54]
 [23 68 51 69 23 66 54 47 23 13]
 [56 15 68 68 77 25 37 23 51 37]
 [23 34 37 37 51 23 66 51 81 23]
 [74 55 16 25 37 68 80 30  5 23]]

y
 [[66 51 20 13 54 47 23 45 53 53]
 [25 37 23 13 66 51 13 23 51 13]
 [37 81 23 25 47 23 51 23 82 25]
 [23 13 66 54 23 56 66 77 54 82]
 [68 51 69 23 66 54 47 23 13 54]
 [15 68 68 77 25 37 23 51 37 81]
 [34 37 37 51 23 66 51 81 23 68]
 [55 16 25 37 68 80 30  5 23 78]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 although the exact numbers may be different. Check to make sure the data is shifted over one step for `y`.

---
## Defining the network with PyTorch

Below is where you'll define the network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `drop_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [33]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [48]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        self.lstm = nn.LSTM(len(self.chars),self.n_hidden,self.n_layers,
                           dropout=self.drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(self.drop_prob)
        
        self.fc = nn.Linear(self.n_hidden, len(self.chars))
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        x,hidden = self.lstm(x,hidden)
        
        x = self.dropout(x)
        
        x = x.reshape(-1, self.n_hidden)
        
        out = self.fc(x)
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [35]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [55]:
## TODO: set your model hyperparameters
# define and print the net
n_hidden=512 #My gpu cannot train more than 512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


### Set your training hyperparameters!

In [56]:
batch_size = 64
seq_length = 100

# Note: It seems 20 epochs is enough
n_epochs = 40 # start small if you are just testing initial behavior


# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/40... Step: 10... Loss: 3.2443... Val Loss: 3.1990
Epoch: 1/40... Step: 20... Loss: 3.1525... Val Loss: 3.1369
Epoch: 1/40... Step: 30... Loss: 3.1421... Val Loss: 3.1295
Epoch: 1/40... Step: 40... Loss: 3.1440... Val Loss: 3.1259
Epoch: 1/40... Step: 50... Loss: 3.1276... Val Loss: 3.1253
Epoch: 1/40... Step: 60... Loss: 3.1361... Val Loss: 3.1236
Epoch: 1/40... Step: 70... Loss: 3.1323... Val Loss: 3.1215
Epoch: 1/40... Step: 80... Loss: 3.1258... Val Loss: 3.1178
Epoch: 1/40... Step: 90... Loss: 3.0987... Val Loss: 3.1044
Epoch: 1/40... Step: 100... Loss: 3.0919... Val Loss: 3.0756
Epoch: 1/40... Step: 110... Loss: 3.0029... Val Loss: 3.0051
Epoch: 1/40... Step: 120... Loss: 3.0076... Val Loss: 2.9875
Epoch: 1/40... Step: 130... Loss: 2.9404... Val Loss: 2.9290
Epoch: 1/40... Step: 140... Loss: 2.8252... Val Loss: 2.8220
Epoch: 1/40... Step: 150... Loss: 2.7368... Val Loss: 2.7457
Epoch: 1/40... Step: 160... Loss: 2.6405... Val Loss: 2.6502
Epoch: 1/40... Step: 170... Loss:

Epoch: 5/40... Step: 1350... Loss: 1.4837... Val Loss: 1.4961
Epoch: 5/40... Step: 1360... Loss: 1.5108... Val Loss: 1.4922
Epoch: 5/40... Step: 1370... Loss: 1.4490... Val Loss: 1.4894
Epoch: 5/40... Step: 1380... Loss: 1.4990... Val Loss: 1.4928
Epoch: 5/40... Step: 1390... Loss: 1.4516... Val Loss: 1.4917
Epoch: 6/40... Step: 1400... Loss: 1.4770... Val Loss: 1.4919
Epoch: 6/40... Step: 1410... Loss: 1.4756... Val Loss: 1.4886
Epoch: 6/40... Step: 1420... Loss: 1.4992... Val Loss: 1.4813
Epoch: 6/40... Step: 1430... Loss: 1.4765... Val Loss: 1.4847
Epoch: 6/40... Step: 1440... Loss: 1.4283... Val Loss: 1.4828
Epoch: 6/40... Step: 1450... Loss: 1.4276... Val Loss: 1.4791
Epoch: 6/40... Step: 1460... Loss: 1.4553... Val Loss: 1.4761
Epoch: 6/40... Step: 1470... Loss: 1.4566... Val Loss: 1.4740
Epoch: 6/40... Step: 1480... Loss: 1.4160... Val Loss: 1.4761
Epoch: 6/40... Step: 1490... Loss: 1.4512... Val Loss: 1.4709
Epoch: 6/40... Step: 1500... Loss: 1.3672... Val Loss: 1.4685
Epoch: 6

Epoch: 10/40... Step: 2670... Loss: 1.2616... Val Loss: 1.3443
Epoch: 10/40... Step: 2680... Loss: 1.3009... Val Loss: 1.3468
Epoch: 10/40... Step: 2690... Loss: 1.2532... Val Loss: 1.3435
Epoch: 10/40... Step: 2700... Loss: 1.2551... Val Loss: 1.3386
Epoch: 10/40... Step: 2710... Loss: 1.2857... Val Loss: 1.3437
Epoch: 10/40... Step: 2720... Loss: 1.3082... Val Loss: 1.3397
Epoch: 10/40... Step: 2730... Loss: 1.2755... Val Loss: 1.3429
Epoch: 10/40... Step: 2740... Loss: 1.3085... Val Loss: 1.3371
Epoch: 10/40... Step: 2750... Loss: 1.2754... Val Loss: 1.3459
Epoch: 10/40... Step: 2760... Loss: 1.2169... Val Loss: 1.3430
Epoch: 10/40... Step: 2770... Loss: 1.2779... Val Loss: 1.3411
Epoch: 10/40... Step: 2780... Loss: 1.3019... Val Loss: 1.3382
Epoch: 10/40... Step: 2790... Loss: 1.3437... Val Loss: 1.3411
Epoch: 11/40... Step: 2800... Loss: 1.2470... Val Loss: 1.3387
Epoch: 11/40... Step: 2810... Loss: 1.2964... Val Loss: 1.3364
Epoch: 11/40... Step: 2820... Loss: 1.2706... Val Loss:

Epoch: 15/40... Step: 3980... Loss: 1.1667... Val Loss: 1.2985
Epoch: 15/40... Step: 3990... Loss: 1.1677... Val Loss: 1.2978
Epoch: 15/40... Step: 4000... Loss: 1.2105... Val Loss: 1.2916
Epoch: 15/40... Step: 4010... Loss: 1.1776... Val Loss: 1.2914
Epoch: 15/40... Step: 4020... Loss: 1.1717... Val Loss: 1.2952
Epoch: 15/40... Step: 4030... Loss: 1.1632... Val Loss: 1.2899
Epoch: 15/40... Step: 4040... Loss: 1.1843... Val Loss: 1.2912
Epoch: 15/40... Step: 4050... Loss: 1.2084... Val Loss: 1.2948
Epoch: 15/40... Step: 4060... Loss: 1.1722... Val Loss: 1.2904
Epoch: 15/40... Step: 4070... Loss: 1.1936... Val Loss: 1.2928
Epoch: 15/40... Step: 4080... Loss: 1.1744... Val Loss: 1.2969
Epoch: 15/40... Step: 4090... Loss: 1.1695... Val Loss: 1.2922
Epoch: 15/40... Step: 4100... Loss: 1.1300... Val Loss: 1.2924
Epoch: 15/40... Step: 4110... Loss: 1.1710... Val Loss: 1.2850
Epoch: 15/40... Step: 4120... Loss: 1.2011... Val Loss: 1.2939
Epoch: 15/40... Step: 4130... Loss: 1.2286... Val Loss:

Epoch: 19/40... Step: 5290... Loss: 1.1118... Val Loss: 1.2782
Epoch: 19/40... Step: 5300... Loss: 1.1338... Val Loss: 1.2731
Epoch: 20/40... Step: 5310... Loss: 1.1405... Val Loss: 1.2793
Epoch: 20/40... Step: 5320... Loss: 1.1740... Val Loss: 1.2741
Epoch: 20/40... Step: 5330... Loss: 1.1290... Val Loss: 1.2806
Epoch: 20/40... Step: 5340... Loss: 1.1251... Val Loss: 1.2766
Epoch: 20/40... Step: 5350... Loss: 1.1047... Val Loss: 1.2808
Epoch: 20/40... Step: 5360... Loss: 1.0931... Val Loss: 1.2778
Epoch: 20/40... Step: 5370... Loss: 1.1380... Val Loss: 1.2778
Epoch: 20/40... Step: 5380... Loss: 1.1274... Val Loss: 1.2739
Epoch: 20/40... Step: 5390... Loss: 1.1206... Val Loss: 1.2835
Epoch: 20/40... Step: 5400... Loss: 1.1148... Val Loss: 1.2766
Epoch: 20/40... Step: 5410... Loss: 1.1120... Val Loss: 1.2766
Epoch: 20/40... Step: 5420... Loss: 1.0999... Val Loss: 1.2734
Epoch: 20/40... Step: 5430... Loss: 1.1506... Val Loss: 1.2675
Epoch: 20/40... Step: 5440... Loss: 1.1335... Val Loss:

Epoch: 24/40... Step: 6600... Loss: 1.0965... Val Loss: 1.2714
Epoch: 24/40... Step: 6610... Loss: 1.1234... Val Loss: 1.2626
Epoch: 24/40... Step: 6620... Loss: 1.1083... Val Loss: 1.2642
Epoch: 24/40... Step: 6630... Loss: 1.1100... Val Loss: 1.2641
Epoch: 24/40... Step: 6640... Loss: 1.1052... Val Loss: 1.2680
Epoch: 24/40... Step: 6650... Loss: 1.1070... Val Loss: 1.2706
Epoch: 24/40... Step: 6660... Loss: 1.1190... Val Loss: 1.2668
Epoch: 24/40... Step: 6670... Loss: 1.0747... Val Loss: 1.2656
Epoch: 24/40... Step: 6680... Loss: 1.1123... Val Loss: 1.2684
Epoch: 24/40... Step: 6690... Loss: 1.0862... Val Loss: 1.2635
Epoch: 25/40... Step: 6700... Loss: 1.0846... Val Loss: 1.2669
Epoch: 25/40... Step: 6710... Loss: 1.1107... Val Loss: 1.2693
Epoch: 25/40... Step: 6720... Loss: 1.0953... Val Loss: 1.2696
Epoch: 25/40... Step: 6730... Loss: 1.1022... Val Loss: 1.2781
Epoch: 25/40... Step: 6740... Loss: 1.1412... Val Loss: 1.2741
Epoch: 25/40... Step: 6750... Loss: 1.0666... Val Loss:

Epoch: 29/40... Step: 7910... Loss: 1.0674... Val Loss: 1.2653
Epoch: 29/40... Step: 7920... Loss: 1.0682... Val Loss: 1.2684
Epoch: 29/40... Step: 7930... Loss: 1.0729... Val Loss: 1.2677
Epoch: 29/40... Step: 7940... Loss: 1.1174... Val Loss: 1.2649
Epoch: 29/40... Step: 7950... Loss: 1.0879... Val Loss: 1.2675
Epoch: 29/40... Step: 7960... Loss: 1.0822... Val Loss: 1.2653
Epoch: 29/40... Step: 7970... Loss: 1.0803... Val Loss: 1.2655
Epoch: 29/40... Step: 7980... Loss: 1.0948... Val Loss: 1.2623
Epoch: 29/40... Step: 7990... Loss: 1.0929... Val Loss: 1.2691
Epoch: 29/40... Step: 8000... Loss: 1.0666... Val Loss: 1.2690
Epoch: 29/40... Step: 8010... Loss: 1.0795... Val Loss: 1.2651
Epoch: 29/40... Step: 8020... Loss: 1.0929... Val Loss: 1.2619
Epoch: 29/40... Step: 8030... Loss: 1.1192... Val Loss: 1.2648
Epoch: 29/40... Step: 8040... Loss: 1.0659... Val Loss: 1.2591
Epoch: 29/40... Step: 8050... Loss: 1.0843... Val Loss: 1.2628
Epoch: 29/40... Step: 8060... Loss: 1.0694... Val Loss:

Epoch: 34/40... Step: 9220... Loss: 1.0661... Val Loss: 1.2659
Epoch: 34/40... Step: 9230... Loss: 1.0675... Val Loss: 1.2602
Epoch: 34/40... Step: 9240... Loss: 1.0432... Val Loss: 1.2680
Epoch: 34/40... Step: 9250... Loss: 1.0461... Val Loss: 1.2711
Epoch: 34/40... Step: 9260... Loss: 1.0322... Val Loss: 1.2649
Epoch: 34/40... Step: 9270... Loss: 1.0750... Val Loss: 1.2715
Epoch: 34/40... Step: 9280... Loss: 1.0203... Val Loss: 1.2725
Epoch: 34/40... Step: 9290... Loss: 1.0546... Val Loss: 1.2648
Epoch: 34/40... Step: 9300... Loss: 1.0474... Val Loss: 1.2708
Epoch: 34/40... Step: 9310... Loss: 1.0374... Val Loss: 1.2672
Epoch: 34/40... Step: 9320... Loss: 1.0014... Val Loss: 1.2683
Epoch: 34/40... Step: 9330... Loss: 1.1085... Val Loss: 1.2694
Epoch: 34/40... Step: 9340... Loss: 1.0965... Val Loss: 1.2646
Epoch: 34/40... Step: 9350... Loss: 1.0429... Val Loss: 1.2647
Epoch: 34/40... Step: 9360... Loss: 1.0557... Val Loss: 1.2635
Epoch: 34/40... Step: 9370... Loss: 1.0468... Val Loss:

Epoch: 38/40... Step: 10520... Loss: 1.0571... Val Loss: 1.2668
Epoch: 38/40... Step: 10530... Loss: 1.0644... Val Loss: 1.2733
Epoch: 38/40... Step: 10540... Loss: 1.0190... Val Loss: 1.2649
Epoch: 38/40... Step: 10550... Loss: 1.0527... Val Loss: 1.2613
Epoch: 38/40... Step: 10560... Loss: 1.0260... Val Loss: 1.2639
Epoch: 38/40... Step: 10570... Loss: 1.0658... Val Loss: 1.2638
Epoch: 38/40... Step: 10580... Loss: 1.0843... Val Loss: 1.2724
Epoch: 38/40... Step: 10590... Loss: 1.0536... Val Loss: 1.2706
Epoch: 38/40... Step: 10600... Loss: 1.0590... Val Loss: 1.2704
Epoch: 39/40... Step: 10610... Loss: 1.0576... Val Loss: 1.2683
Epoch: 39/40... Step: 10620... Loss: 1.0702... Val Loss: 1.2682
Epoch: 39/40... Step: 10630... Loss: 1.0072... Val Loss: 1.2679
Epoch: 39/40... Step: 10640... Loss: 1.0390... Val Loss: 1.2721
Epoch: 39/40... Step: 10650... Loss: 1.0288... Val Loss: 1.2648
Epoch: 39/40... Step: 10660... Loss: 1.0395... Val Loss: 1.2720
Epoch: 39/40... Step: 10670... Loss: 1.0

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [57]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [58]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [59]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [60]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna was a man,
sitting in a white lady and waistcoat, began to go away. The same tense,
work for a much for his whole. The day before the sense horternal for
some observations of the way, but he went ovirton to the crowd, but the
man who seemed in her eyes from this mistake and ashonated, and was
beginning.

His father-and there was nothing in his husband.

As they were seeing his soul in the pilto as she said that his memory at
the point he was supported at his brideger in a side peasant with his
solation to a conversation as he would have been seening the peasants
often.

"Yes, I want to make me. That would say, but who did you know they certainly
become seeing the chair in the meadow? Though, as he said, when he cannot be
able to stay to such a peasant woman..."

"I am very will to send him and himself. Ah!" said Vassily Lukitch, shutifuing
it.

Alexey Alexandrovitch, as he was stating in the drawing room the calm in a
setting of a short littlo gown. The servants had assembled him 

In [61]:
# To compare
print(sample(net, 2000, top_k=5, prime="And Levin said"))

And Levin said
to her, took the bride's face, and turned out of the conversation.

"I'm not married," said Anna, as though it was all about ansting at him
to go on with her; he did not laugh, and her son, with a sharp will believed
in them.

"All right, there was the manager, I'll send a little. But it cunnot any man we
suppose, but it was now that. He went to their pirturs and wants to be
ashed to a strange solicitor and her husband's sake; but there's no possible
seeing it only one thing."

"Well, as you mustn't tries."

"No; then."

"They've confined her soul; it's a means to start, shall be one, but
I heard you," he said with a smile, "that that way. They're so mere
himself. I'm not there."

"Will you take him to the lady of a master shall not let mis of a
match and be a spring or an irritable, to take me to his streams. Has
always to start. But to be sure, and that I while it!"

She was surposed with the same stand. He walked into the door the prince
would gath on and to his fine 

## Loading a checkpoint

In [62]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [63]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said
to him. He had thought of what they were to be merrily, wanted to gain a man
again, and was already standing in haste to his brother's hands, and
this there was no misual honsthants to talk about him. He had been delighted,
this important misfortune he should not be a moment.

"Why, a discombod matter," said Vronsky, standing up in the chush of the
precent charm, which happened was shaking hasring, and hid her hand, thought
any sort of day after the peasants of his most convention with his face
in the sake of a love, she was not to give him the stead the ball with
whom he was not simply anxoored. There was no think of this friend, that his
brother with them that is not a look of little morning. The second accompaniment
was said; but he was told it would not be studiously trying to be thought.

"You come away. If you can't get in from the course of miles, this
impossible to do them."

Anna was so successful, and had never said this, and had thinked of
himself. This calmer